In [43]:
data_dir = "./Data/Weather/"
%matplotlib inline
!mkdir -p $data_dir
!ls $data_dir

STAT.pickle           SampleStations.pickle


### Downloading Pickled data from S3
If `SampleStations.pickle` is not in the directory, get it using the following command

In [12]:
!curl -o $data_dir/SampleStations.pickle http://mas-dse-open.s3.amazonaws.com/Weather/SampleStations.pickle

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  398M  100  398M    0     0  23.5M      0  0:00:16  0:00:16 --:--:-- 26.4M


### Plot Reconstructions

From each measurement, we take 2 sample stations with low residual error and 2 sample stations with high residual error and plot the following:

* Original Data Vector
* Mean Vector
* Reconstructed Data Vector using mean and top 1 eigen vectors
* Reconstructed Data Vector using mean and top 2 eigen vectors
* Reconstructed Data Vector using mean and top 3 eigen vectors

### Read and Preprocess data

Read data from the pickle files `SampleStations.pickle` and `STAT.pickle`

* `SampleStations.pickle` contains information about the 6 measurements taken from some sample stations. You have been provided the code to process this file. It converts the seemingly complicated structure into a list of lists where each sublist has the following information:

`['station','measurement','year','1', '2', '3', ...... , '365']`

*  station - The station ID
*  measurement - One of the 6 types of measurements
*  year - The year in which the measurements were recorded
*  1-365 - The actual value of measurement for each day of the year



* `STAT.pickle` contains statistics about the weather data for each of the 6 measurements and its description.

In [44]:
import pickle
Data=pickle.load(open('./Data/Weather/SampleStations.pickle','r'))
STAT,STAT_description=pickle.load(open('./Data/Weather/STAT.pickle','r'))

In [45]:
FlatData=[]
for station in Data:
    stationname=station[0]
    for measurements in station[1]:
        measurement,year=measurements[0]
        yeardata=list(measurements[1])
        rowData=[stationname]+[measurement]+[year]+yeardata
        FlatData.append(rowData)

### Define Reconstruction Function

You need to plot reconstructions for two stations with low reconstruction error and two stations with high reconstruction error. To do this, you will need to do the following:

1. Calculate the reconstruction error for the data vector reconstructed using mean and top-3 eigenvectors.
2. Remove the ones for which reconstruction error is NaN.
3. Choose two with the lowest and two with the highest reconstruction error for plotting.

In [26]:
def create_reconstructions(m):  
    
    ## Put your code for computating reconstructions here
    
    plt.figure(figsize=(30,20),dpi=2000)
    offset=1
    # lower contains reference to the 2 data vectors with lowest reconstruction error
    for l in lower:  
        subplot(2,2,j)
        offset+=1
        ## Put your code for plotting reconstruction for stations with low residual error
     
    # higher contains reference to the 2 data vectors with highest reconstruction error
    for l in higher:
        subplot(2,2,j)
        offset+=1
        ## Put your code for plotting reconstruction for stations with high residual error
        

In [8]:
import numpy as np
from numpy import isnan
columns=4   # each column corresponds to an eigenvector of the reconstruction

def create_reconstructions(m):  
    Prod={}
    #Compute the projection of (Data-Mean) on the top k eigenvectors
    for k in range(1,columns):
        Eigen=np.matrix(STAT[m]['eigvec'][:,:k])
        matrix=np.matrix(m_df[m].ix[:,3:368])-STAT[m]['Mean']
        matrix[isnan(matrix)]=0
        Prod[k]=matrix*Eigen
    
    Reconstructed={}
    for k in range(1,columns):
        Eigen=np.matrix(STAT[m]['eigvec'][:,:k])
        Reconstructed[k]=array(Eigen*Prod[k].transpose()+STAT[m]['Mean'][:,np.newaxis])
        
    original=np.array(m_df[m].ix[:,3:368])
    residue_list=[]
    #print len(Reconstructed),shape(original)
    for i in range(len(Reconstructed[1][0])):
        vec_diff=original[i]-Reconstructed[3][:,i]
        residue=np.linalg.norm(vec_diff)
        residue_list.append((residue,i))
    residue_list=[x for x in residue_list if not math.isnan(x[0])]
    residue_list=sorted(residue_list)
    
    lower=[i for (r,i) in residue_list[:2]]
    upper=[i for (r,i) in residue_list[-2:]]
    
    yeardays=[i for i in (1,366)]
    plt.figure(figsize=(20,30),dpi=300)
    j=1
    c=0
    for l in lower:
        subplot(4,2,j)
        j+=1
        c+=1
        plot(original[l])
        plot(STAT[m]['Mean'])
        plot(Reconstructed[1][:,l])
        plot(Reconstructed[2][:,l])
        plot(Reconstructed[3][:,l])
        title('#' + str(c) + ' Sample for ' + m + ' (low residual error)')
        xlim([0,365])
        #Min,Max=min(original[l]),max(original[l])
        #print Min,Max
        #ylim(Min,Max)
        legend(['original','Mean','1','2','3'],loc=2)
    
    c=0
    for l in upper:
        subplot(4,2,j)
        j+=1
        c+=1
        plot(original[l])
        plot(STAT[m]['Mean'])
        plot(Reconstructed[1][:,l])
        plot(Reconstructed[2][:,l])
        plot(Reconstructed[3][:,l])
        title('#' + str(c) + ' Sample for ' + m + ' (high residual error)')
        xlim([0,365])
        legend(['original','Mean','1','2','3'],loc=2)


In [9]:
for m in ['TMAX','SNWD']:
    print 'Reconstruction Plots for '+ m
    create_reconstructions(m)

Reconstruction Plots for TMAX
Reconstruction Plots for SNWD
